# Analyze Historical Calendar Events

In [1]:
import os
import sys
import json
import csv
from datetime import datetime
from importlib import import_module

import pandas as pd

sys.path.append(os.path.abspath('../src'))
from gcal.util import get_client
from event import from_gcal_events

In [2]:
USERNAME = 'zack@plenty.ag'
START_DT = datetime(2016, 6, 13)
END_DT = datetime.utcnow()

START_DT_STR = START_DT.isoformat() + 'Z'
END_DT_STR = END_DT.isoformat() + 'Z'

In [3]:
CACHE_PATH = 'HIST_CACHE.csv'

In [9]:
def load_events():
    if not os.path.exists(CACHE_PATH):
        calendar = get_client()

        all_results = []
        req = calendar.events().list(
            calendarId='zack@plenty.ag',
            timeMin=START_DT_STR,
            timeMax=END_DT_STR,
            singleEvents=True,
            orderBy='startTime'
        )
        while req:
            res = req.execute()
            all_results += res['items']
            req = calendar.events().list_next(req, res)
            print(all_results[-1].get('start', {}).get('dateTime'))

        headers = set()
        for e in all_results:
            headers |= e.keys()

        with open(CACHE_PATH, 'w+') as fout:
            writer = csv.DictWriter(fout, fieldnames=headers)
            writer.writeheader()
            for e in all_results:
                writer.writerow(e)

    return pd.read_csv(CACHE_PATH)

gcal_events = load_events()

In [10]:
print('Days:', (END_DT - START_DT).days)
print('Evts:', len(gcal_events))
print('Cols:', gcal_events.columns)

Days: 1258
Evts: 5395
Cols: Index(['originalStartTime', 'status', 'reminders', 'creator', 'id', 'kind',
       'description', 'privateCopy', 'recurringEventId', 'updated',
       'attachments', 'etag', 'guestsCanInviteOthers', 'transparency',
       'created', 'iCalUID', 'guestsCanSeeOtherGuests', 'attendees',
       'location', 'guestsCanModify', 'organizer', 'extendedProperties',
       'visibility', 'end', 'start', 'conferenceData', 'sequence', 'summary',
       'htmlLink', 'hangoutLink'],
      dtype='object')


In [55]:
event_models = from_gcal_events(gcal_events, 'zack.swafford@plenty.ag')
events = pd.DataFrame([e.serialize() for e in event_models])